## 使用预训练的词向量

In [1]:
import torch
import torchtext.vocab as vocab

vocab.pretrained_aliases.keys()

dict_keys(['charngram.100d', 'fasttext.en.300d', 'fasttext.simple.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d'])

In [2]:
[key for key in vocab.pretrained_aliases.keys() if "glove" in key]

['glove.42B.300d',
 'glove.840B.300d',
 'glove.twitter.27B.25d',
 'glove.twitter.27B.50d',
 'glove.twitter.27B.100d',
 'glove.twitter.27B.200d',
 'glove.6B.50d',
 'glove.6B.100d',
 'glove.6B.200d',
 'glove.6B.300d']

In [3]:
cache_dir = "../data/glove"
# glove = vocab.pretrained_aliases["glove.6B.50d"](cache=cache_dir)
glove = vocab.GloVe(name='6B', dim=50, cache=cache_dir) # 与上面等价

../data/glove\glove.6B.zip: 862MB [41:15, 348kB/s]                                                                     
100%|██████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:11<00:00, 34675.40it/s]


返回的实例主要有以下三个属性：

- stoi: 词到索引的字典：
- itos: 一个列表，索引到词的映射；
- vectors: 词向量。

In [4]:
print("一共包含%d个词。" % len(glove.stoi))

一共包含400000个词。


In [5]:
glove.stoi['beautiful'], glove.itos[3366]

(3366, 'beautiful')

## 应用预训练词向量

### 求近义词

In [6]:
def knn(W, x, k):
    # 添加的1e-9是为了数值稳定性
    cos = torch.matmul(W, x.view((-1,))) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

In [7]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.vectors,
                    embed.vectors[embed.stoi[query_token]], k+1)
    for i, c in zip(topk[1:], cos[1:]):  # 除去输入词
        print('cosine sim=%.3f: %s' % (c, (embed.itos[i])))

In [8]:
get_similar_tokens('chip', 3, glove)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [9]:
get_similar_tokens('baby', 3, glove)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [10]:
get_similar_tokens('beautiful', 3, glove)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


### 求类比词

除了求近义词以外，我们还可以使用预训练词向量求词与词之间的类比关系。例如，“man”（男人）: “woman”（女人）:: “son”（儿子） : “daughter”（女儿）是一个类比例子：“man”之于“woman”相当于“son”之于“daughter”。求类比词问题可以定义为：对于类比关系中的4个词 $a:b::c:d$，给定前3个词$a$、$b$和$c$，求$d$。设词$w$的词向量为$vec(w)$。求类比词的思路是，搜索与$vec(c)+vec(b)−vec(a)$的结果向量最相似的词向量。

In [11]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = [embed.vectors[embed.stoi[t]] for t in [token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.vectors, x, 1)
    return embed.itos[topk[0]]

In [12]:
get_analogy('man', 'woman', 'son', glove) # 'daughter'

'daughter'

In [13]:
get_analogy('beijing', 'china', 'tokyo', glove) # 'japan'

'japan'

In [14]:
get_analogy('do', 'did', 'go', glove) # 'went'

'went'